# Pressure-velocity boundary conditions for cavity flow

This section looks at cavity flow again, but with a recalibration of the pressure boundary conditions.

The Navier-Stokes equations in two dimensions, momentum and the Poisson pressure equation, are:

$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu \left(\frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2} \right)$$

$$\frac{\partial v}{\partial t}+u\frac{\partial v}{\partial x}+v\frac{\partial v}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial y}+\nu\left(\frac{\partial^2 v}{\partial x^2}+\frac{\partial^2 v}{\partial y^2}\right)$$

$$\frac{\partial^2 p}{\partial x^2}+\frac{\partial^2 p}{\partial y^2} = -\rho\left(\frac{\partial u}{\partial x}\frac{\partial u}{\partial x}+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\frac{\partial v}{\partial y}\frac{\partial v}{\partial y} \right)$$

Again, we assume that the current flow field is not divergence-free in the numerical schemes: $\nabla \bf u^{n+1/2} \neq 0$, so this assumption is used to calculate the pressure field:

$$\nabla^2 p^{n+1}=\frac{\rho}{\Delta t}\nabla\cdot\bf{u}^{n+1/2}-\rho\left(\left(\frac{\partial u}{\partial x}\right)^2+2\frac{\partial u}{\partial y}\frac{\partial v}{\partial x}+\left(\frac{\partial u}{\partial y}\right)^2\right)$$

which is the Poisson pressure equation, but maintaining the numerical divergence term that is zero in the continuous formulation. This discretizes as:

$$
\frac{p_{i+1,j}^{n}-2p_{i,j}^{n}+p_{i-1,j}^{n}}{\Delta x^2}+\frac{p_{i,j+1}^{n}-2*p_{i,j}^{n}+p_{i,j-1}^{n}}{\Delta y^2} 
=\rho\left[\frac{1}{\Delta t}\left(\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}+\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right)\right.-\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}\frac{u_{i+1,j}-u_{i-1,j}}{2\Delta x}
- \ 2\frac{u_{i,j+1}-u_{i,j-1}}{2\Delta y}\frac{v_{i+1,j}-v_{i-1,j}}{2\Delta x}
-\left.\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\frac{v_{i,j+1}-v_{i,j-1}}{2\Delta y}\right]
\quad 
$$


## Pressure boundary conditions

The pressure boundary conditions will be carefully evaluated. The NS momentum equation can be written as:

$$ \nabla p= \rho \left( -\frac{\partial{\bf{u}}}{\partial t}-\bf{u}\cdot\nabla\bf{u}+\nu\nabla^2\bf{u}\right)\quad$$

If we take the dot product of each side with a normal vector of the boundary, we obtain:

$$\bf{n}\cdot\nabla p=\rho \bf{n}\cdot\left( -\frac{\partial{\bf u}}{\partial t} -\bf{u}\cdot\nabla\bf{u}+\nu\nabla^2\vb{u}\right)\quad$$

For a rigid boundary, the velocity at the boundary is zero. On the $y=0$ boundary, for example, this would give:

$$\hat{\bf j}\cdot\left(\frac{\partial p}{\partial x}\hat{\bf i} + \frac{\partial p}{\partial y}\hat{\bf j}\right) = \rho\hat{\bf j}\cdot\left(-\frac{\partial u}{\partial t}\hat{\bf i} - \frac{\partial v}{\partial t} \hat{\bf j} - \left(u\hat{\bf i} + v\hat{\bf j}\right)\cdot\left(\frac{\partial u}{\partial x}\hat{\bf i} + \frac{\partial v}{\partial y}\hat{\bf j}\right) + \nu\left(\frac{\partial^2 u}{\partial x^2}\hat{\bf i} + \frac{\partial^2 v}{\partial y^2}\hat{\bf j}\right)\right)$$ 

Assuming steady flow, the above equation simplifies to:

$$\frac{\partial p}{\partial y} = \rho\nu\frac{\partial^2 v}{\partial y^2}$$

So, there may be a pressure gradient on a rigid boundary, depending on the curvature of the velocity field in space there.

To find the pressure gradient on the boundary, we must find the second derivative of the velocity on the boundary. The normal central difference scheme is problematic there, because it requires evaluation past the boundary. Instead, we can use the scheme:

$$\left.\frac{\partial^2 v}{\partial y^2}\right|_{i,j=0}=\frac{1}{\Delta y^2}\left(v_{i,j=0}-2 v_{i,j=1}+v_{i,j=2}\right)$$


In [1]:
import matplotlib.pyplot as plt
from matplotlib import cm 
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
def pressPoisson(p, dx, dy, rho, nu, u, v):
    pn = np.empty_like(p)
    pn = np.copy(p)
    
    # For readibility, the term in square brackets is below